In [9]:
import numpy as np
import numba as nb

In [2]:
a = np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [3]:
a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [4]:
from wireless import hamming

In [5]:
N_iter = 2
u_array = np.random.randint(2, size=(N_iter, 4))

In [6]:
u_array

array([[0, 1, 1, 1],
       [1, 1, 0, 0]])

In [7]:
hamming.encode_array_n_hamming(u_array)

/home/sjkim/data/github_download/jamespy_py3/wireless/hamming.py:1041: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "encode_array_n_hamming" failed type inference due to: Invalid use of Function(<function zeros_like at 0x7f77382320e0>) with argument(s) of type(s): (array(int64, 2d, C), shape=UniTuple(int64 x 2))
 * parameterized
In definition 0:
    TypeError: typer() got an unexpected keyword argument 'shape'
    raised from /home/sjkim/anaconda3/envs/tf2/lib/python3.7/site-packages/numba/typing/templates.py:283
In definition 1:
    TypeError: typer() got an unexpected keyword argument 'shape'
    raised from /home/sjkim/anaconda3/envs/tf2/lib/python3.7/site-packages/numba/typing/templates.py:283
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<function zeros_like at 0x7f77382320e0>)
[2] During: typing of call at /home/sjkim/data/g

array([[0, 1, 1, 1, 3, 2, 2],
       [1, 1, 0, 0, 1, 1, 2]])

In [10]:
G = np.array([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [0,1,1,1],
              [1,0,1,1],
              [1,1,0,1]]) 
u_array = np.random.randint(2, size=(10, 4))
x_array = np.zeros_like(u_array, shape=(u_array.shape[0], G.shape[0]))

In [11]:
x_array.shape

(10, 7)

In [10]:
@nb.jit
def encode_array_n_hamming(u_array):
    G = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1],
                  [0,1,1,1],
                  [1,0,1,1],
                  [1,1,0,1]])    
    x_array = np.zeros_like(u_array, shape=(u_array.shape[0], G.shape[0]))
    for i in range(x_array.shape[0]):
        for j in range(x_array.shape[1]):
            x_array[i,j] = np.sum(u_array[i,:] * G[j,:])
    return x_array

In [70]:
@nb.jit
def _encode_array_n_hamming(u_array):
    G = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1],
                  [0,1,1,1],
                  [1,0,1,1],
                  [1,1,0,1]])    
    x_array = np.zeros(shape=(u_array.shape[0], G.shape[0]), dtype=u_array.dtype)
    for i in range(x_array.shape[0]):
        for j in range(x_array.shape[1]):
            x_array[i,j] = np.mod(np.sum(u_array[i,:] * G[j,:]),2)    
    return x_array

def encode_array_n_hamming(u_array):
    G = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1],
                  [0,1,1,1],
                  [1,0,1,1],
                  [1,1,0,1]])    
    return np.mod(np.dot(u_array, G.T),2)

def channel_numpy_awgn(x_array, SNRdB):
    """
    출력을 (0,1) --> (1,-1)로 바꾸고 가우시안 노이즈를 더함.
    """
    #y_array = np.zeros(x_array.shape, dtype=nb.float_)
    SNR = np.power(10, SNRdB/10)
    noise_sig = 1/np.sqrt(SNR)
    n_array = np.random.normal(0.0, noise_sig, size=x_array.shape)
    y_array = 1.0 - x_array*2 + n_array
    return y_array   

def decode_array_n_hamming(y_array, K_code:int=4):    
    G = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1],
                  [0,1,1,1],
                  [1,0,1,1],
                  [1,1,0,1]])        
    a = np.arange(0,2**K_code,dtype='uint8')
    m = np.unpackbits(a.reshape(-1,1), axis=1)[:,-4:]
    # m을 uint8에서 int 즉 int64로 바꾸기 위한 코드임.
    m = np.dot(m, np.eye(K_code, dtype=int))
    # print(m.dtype)
    x = 1.0 - 2.0*np.mod(np.dot(m, G.T),2) # 16 x 7
    r_array = np.dot(y_array, x.T)
    argmax_r_arry = np.argmax(r_array,axis=1)
    ud_array = m[argmax_r_arry,:]
    return ud_array
    #return argmax_r_arry

u_array = np.random.randint(2, size=(5, 4))
x_array = encode_array_n_hamming(u_array)
y_array = channel_numpy_awgn(x_array, 10)
ud_array = decode_array_n_hamming(y_array)
u_array, ud_array

(array([[0, 0, 0, 0],
        [1, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 1]]),
 array([[0, 0, 0, 0],
        [1, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 1]]))

In [64]:
np.max(r_array,axis=1)

array([8.00300711, 7.43678122])